# Getting Started with RAiDER

**Author**: Jeremy Maurer, David Bekaert, Simran Sangha - Jet Propulsion Laboratory

This notebook provides an overview of how to get started using the RAiDER package for estimating tropospheric RADAR delays, and other functionality included in the **raiderDelay.py** program. We give an example of how to download and process delays using ERA-5 and HRRR weather models for the Los Angeles region. 

In this notebook, we will demonstrate how to:
- Download and install the RAiDER package
- Run RAiDER to generate a grid of delays over the Los Angeles region
- Compare tropospheric delays from the weather model to that obtained from a GNSS station
    
<div class="alert alert-warning">
The initial setup (<b>Prep A</b> section) should be run at the start of the notebook. The overview sections do not need to be run in order. 
</div>

<div class="alert alert-danger">
<b>Potential Errors:</b> 
    
- GDAL uses "HDF5" driver instead of "netCDF/Network Common Data Format." Verify GDAL version >= 3.
- RAiDER needs to be installed to run this notebook
</div>


<div class="alert alert-info">
    <b>Terminology:</b>
    
- *Acquisition*: A single image acquired by a satellite at a particular time
- *Interferogram*: An unwrapped image containing the surface displacement accumulated between two acquisitions.
- *Weather model*: A reanalysis weather product defining temperature, pressure, and humidity on a regular grid in some coordinate system (e.g., at regular lat/lon intervals in the standard EPSG:4326 reference frame).
- *Delay*: The apparent displacement in an interferogram that occurs solely due to changes in weather conditions between two acquisitions.
    </div>
    

## Prep A. Initial setup of the notebook

Below we set up the directory structure for this notebook exercise. In addition, we load the required modules into our python environment using the **`import`** command. We also explicitly enable exceptions for GDAL as this allows us to capture GDAL errors.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

## Defining the home and data directories at the processing location
work_dir = os.path.abspath(os.getcwd())
tutorial_home_dir = os.path.abspath(os.getcwd())
print("Work directory: ", work_dir)
print("Tutorial directory: ", tutorial_home_dir)

# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# Verifying if ARIA-tools is installed correctly
try:
    from RAiDER.delay import tropo_delay
except:
    raise Exception('RAiDER is missing from your PYTHONPATH')
        
os.chdir(work_dir)

Below we define a plotting function that will be used throughout the notebook for plotting GDAL compatible datasets on a map.

In [ ]:
def plot_layer(path_layer, lay_type=None, cmap=None, n_bands=None, **kwargs):
    """ 
        path_layers is a string to the GDAL compatible dataset to be plotted
    """
    
    if not lay_type: 
        lay_type = os.path.dirname(path_layer)
    title = [os.path.basename(lay_type)]
    
    ## get the lon lat bounds
    ds       = gdal.Open(path_layer, gdal.GA_ReadOnly)
    trans    = ds.GetGeoTransform()
    extent   = [trans[0], trans[0] + ds.RasterXSize * trans[1], trans[3] + ds.RasterYSize*trans[5], trans[3]]
    
    ## loading the data
    if not n_bands:
        n_bands  = ds.RasterCount
    lst_arrs = []
    
    for band in range(n_bands):
        raster = ds.GetRasterBand(band+1)
        arr    = raster.ReadAsArray()
        try:
            NoData = raster.GetNoDataValue()
            arr = np.ma.masked_where((arr>1e20) |(arr==NoData),arr )
        except:
            print('Could not find a no-data value...')
            arr = np.ma.masked_where(arr>1e20,arr)
        
        lst_arrs.append(arr)

    ds = None
    if n_bands < 4:
        nrows = 1; ncols = n_bands
    else:
        raise Exception('Number of bands currently unsupported')
        
    
    ## initializing a figure
    fig, axes = plt.subplots(figsize=(12,9), ncols=ncols, nrows=nrows, sharex='col', sharey='row')
    axes = axes if isinstance(axes, np.ndarray) else np.array(axes)
    axe  = axes.ravel() 
    cmap = plt.cm.Greys_r
    cmap.set_under('black')
    
    ## definging the plotting options for differnt layer types
    # Amplitude:
    if lay_type.endswith('amplitude'): 
        # will fix the maximum amplitude bound
        vmin=None
        vmax = 2000 
    # Coherence:
    elif lay_type.endswith('coherence'): 
        # has fixed range between 0-1
        vmin=0
        vmax = 1
    # Incidence angle:
    elif lay_type.endswith('incidenceAngle'): 
        vmin=None
        vmax=None
    # water
    elif lay_type.startswith('water'):
        # no bounds needed will be a 0/1 mask
        vmin=0
        vmax=1
        cmap='Greys'
    # deformation or unwrapped phase
    elif lay_type.startswith('defo'): 
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.coolwarm
    elif lay_type.startswith('terr') or lay_type.startswith('topo'): 
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.terrain
    elif lay_type == 'ENU':
        vmin=None
        vmax=None
        title = ['East', 'North', 'Up']
        fig.subplots_adjust(wspace=0.5)

    else:
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.coolwarm
        
    # plotting the data    
    for i, ax in enumerate(axe):
        im   = ax.imshow(lst_arrs[i], cmap=cmap, vmin=vmin, vmax=vmax, extent=extent)
        divider = make_axes_locatable(ax)
        cax     = divider.append_axes('right', size='5%', pad=0.25)
        if lay_type == 'ENU':
            fig.colorbar(im, cax=cax, format=FuncFormatter(lambda x, y: '{:.3f}'.format(x)))
        elif lay_type.startswith('water'):
            fig.colorbar(im, cax=cax, ticks=[vmin, vmax])
        else:
            fig.colorbar(im, cax=cax)

        ax.set_title(title[i], fontsize=15)
        ax.grid(False)

    axe[0].set_ylabel('latitude', labelpad=15, fontsize=15)
    axe[int(np.floor(n_bands/2))].set_xlabel('longitude', labelpad=15, fontsize=15)

# Obtaining weather model products
## ECMWF products 
1. Create an account on the Copernicus servers [here](https://cds.climate.copernicus.eu/user)
2. Confirm your email, etc.
3. Install the [public API key and client](https://cds.climate.copernicus.eu/api-how-to)
   a. Copy the URL and API key from the webpage into a file in your home directory name ~/.cdsapirc
   b. Install the CDS API using pip: pip install cdsapi
4. You must [accept the license](https://cds.climate.copernicus.eu/cdsapp/#!/terms/licence-to-use-copernicus-products) for each product you wish to download
5. See the test_cdsapi.py script for details of the API. You can test that you can connect to the servers by running the test suite (described below).
## HRRR products 
1. High-resolution rapid refresh (HRRR) weather model data products are generated by [NOAA]() but not archived beyond three days. However a public HRRR archive is available at the University of Utah [archive](home.chpc.utah.edu/~u0553130/Brian_Blaylock/hrrr_FAQ.html). This archive does not require a license agreement


### Download options

ECMWF and HRRR weather model products can be downloaded automatically in RAiDER. Other weather models can be specified manually using the WRF or -wmnetcdf options and described in the docs. 

<div class="alert alert-warning">
<b>Potential download failure:</b> 
ERA-5/ERA-I products require access to the ESA Copernicus servers. If you are unable to download products, ensure that you have registered and have downloaded the public API key, and accepted the license for type of product you wish to download. 
</div>

## Overview of the raiderDelay.py program
<a id='overview'></a>

The **`raiderDelay.py`** program allows for easy downloading and processing of tropospheric weather delays for InSAR correction. Running **`raiderDelay.py`** with the **`-h`** option will show the parameter options. 

Let us explore these options:

In [ ]:
!raiderDelay.py -h

### 1. Date or date list (--date)

This argument is required unless the weather model files are directly specified. The date passed can be either: 
1) a single date, specified in psuedo-ISO 8601 format: 20180101, 2018-01-01, etc. 
2) a comma-deliminated pair of dates, in which case all the dates between the pair listed will be downloaded and processed. E.g., '20180101,20190101'

### 2. Time of day (--time)

This argument is also required unless files are explicitly passed. Specify the time of day to download and process in psuedo-ISO 8601 format: 
1) T020000
2) T02:00:00.000
3) T0200
etc. 

### 3. Line-of-sight vector file (-l/--lineofsight)

This option can be used to specify a line-of-sight file such as those generated as outputs from the ISCE software package for InSAR (github.com/isce-framework/isce2). This should be a two-band GDAL-readable raster file containing the look vector components, with the incidence angle in band 1 and the heading in band 2.

### 4. Statevector file (-s/--statevectors)

This should be an ISCE-derived XML file or a shelve file containing state vectors specifying the orbit of the sensor. 

### Options for Specifying Area/points to process

### 5. Lat/Lon raster files area option (-/--area)

A two-argument option giving the latitude and longitude raster files, such as would be output from the ISCE processor. 

### 6. Bounding box area option (-modelbb/--modelBBOX

This option takes four floats that specify a bounding box (given as North West South East) to be processed. 

### 7.  Station file area option (--station_file)

Instead of a lat/lon file pair or bounding box, a list of lat/lon locations (i.e., stations) can be specified. This should be a .csv file contiaining at least the columns Lat and Lon. 

### DEM (-d/--dem)

The DEM over the area of interest can be specified explicitly, or it will be downloaded on-the-fly. RAiDER will check the default location for a DEM (./geom), so if you download the DEM once it will not be downloaded again. 

### Height levels (--heightlvs)

This option specifies a list of heights, for which the delay will be calculated. The result is a full 3-D cube of delays at the specified lat/lon grid and height levels. 

### Specified weather model files (--wmnetcdf)

Allows for directly passing a list of netcdf files (the appropriate reader must also be specified) to RAiDER. 

## Other input options

### Weather model file directory location (-w/--weatherModelFileLocation)

Specifies a directory to store the original weather model files. If not specified, the default location is ./weather_files.

### Reference integration height (-z/--zref)

This option allows the user to specify the integration height when computing the total delay. The default is 15 km. 

### Output file format (--outformat)

This option is only used if the inputs are rasters or a bounding box, otherwise the output format is fixed (.csv file for station list, HDF5 file for height levels specified). Must be GDAL-compatible. 

### Output file directory (--out)

This specifies the location of the output files. If not specified the default is ./results

### Parallel Computation flag (-p, --no_parallel)

In [ ]:
If specified, do not run in parallel. Off by default. 

### Download the weather model only (--download_only)

If specified, will only download the weather model and do nothing else. 

### Run in verbose mode (-v/--verbose)

Runs the code in verbose mode. Will save the weather model to a pickle file for inspection and create debugging plots.

## WRF-specific options

### WRF model files (--wrfmodelfiles)

A two-argument list of files (out plev) specfied for the WRF model

# RAiDER Readers

Weather model readers in provide the link between the raw weather model data available from HRRR or ECMWF, for example, and the absolute delay calculation. Readers can be added by users to account for other models are custom formats. Here we provide an overview of the WeatherModel class object and requirements for writing one's own reader function. 

## The WeatherModel class

### Functions to be overloaded:
\_fetch: 
- Called by WeatherModel.fetch method
- downloads or loads data from the source files
load_weather: 
- Called by the WeatherModel.load method
- loads data from the raw weather model files and parses it into the WeatherModel format (see below)

### Defining the Reader

The readers are defined with respect to the base WeatherModel class. At minimum, the \_\_init__, \_fetch, Name, and load_weather methods are required. 

A number of important parameters can be defined within the \_\_init__ method, if they are not specified, the defaults listed in the base WeatherModel class will be used.

In [ ]:
from RAiDER.models.weatherModel import WeatherModel

class customModelReader(WeatherModel):
    def __init__(self):
        WeatherModel.__init__(self)
        self._humidityType = 'q'  # can be "q" or "rh"
        self._model_level_type = 'pl' # Default, pressure levels are 'pl'
        self._expver = '0001'
        self._classname = 'hrrr'
        self._dataset = 'hrrr'

        # Tuple of min/max years where data is available. 
        self._valid_range = (datetime.datetime(2016,7,15),"Present")
        self._lag_time = datetime.timedelta(hours=3) # Availability lag time in days

        # model constants
        self._k1 = 0.776  # [K/Pa]
        self._k2 = 0.233 # [K/Pa]
        self._k3 = 3.75e3 # [K^2/Pa]

        # 3 km horizontal grid spacing
        self._lat_res = 3./111
        self._lon_res = 3./111
        self._x_res = 3.
        self._y_res = 3.

        self._Name = 'HRRR'

        # Projections in RAiDER are defined using pyproj (python wrapper around Proj)
        # Parameters of the Lambert Conformal Conic projection
        lon0 = 262.5
        lat0 = 38.5
        lat1 = 38.5
        lat2 = 38.5
        a = 6371229
        b = 6371229
        x0 = 0
        y0 = 0
        p1 = pyproj.Proj('+proj=lcc +lat_1={} +lat_2={} +lat_0={}+lon_0={} +x_0={} +y_0={} +a={} +b={} +units=m +no_defs'.format(lat_1, lat_2, lat_0, lon_0, x0, y0, a, b))
        self._proj = p1
       
    def _fetch(self):
        pass
    def load_weather(self):
        pass

### Required data format

<div class="alert alert-warning">
The '_zs' variable should be topographic height, but the height variable passed with the weather model data is often the geopotential height, which must be converted to topographic height. The WeatherModel class has a helper function for this conversion, which can be called within the custom class as self._get_heights(lats, geo_hgt), where geo_hgt is geopotential height. 
</div>

### Debugging
The WeatherModel class has two built-in plots for debugging purposes, which can be called using 